Create parcel-level zoningmods master file.

In [1]:
import pandas as pd
import numpy as np
import time

today = time.strftime('%Y%m%d')

In [2]:
p10_pba50_FBP_attr = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_FBP_attr_20201110.csv')

print('p10_pba50_FBP_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_FBP_attr.shape[0],
                                                                    len(p10_pba50_FBP_attr.PARCEL_ID.unique())))
print('p10_pba50_FBP_attr fields: {}'.format(list(p10_pba50_FBP_attr)))

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


p10_pba50_FBP_attr has 1956208 rows, 1956208 unique PARCEL_IDs
p10_pba50_FBP_attr fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area']


In [3]:
# create "p10_pba50_EIR_attr", starting with copying "p10_pba50_FBP_attr" 
p10_pba50_EIR_attr = p10_pba50_FBP_attr.copy()
p10_pba50_EIR_attr['PARCEL_ID'] = p10_pba50_EIR_attr['PARCEL_ID'].apply(lambda x: int(round(x)))

# create columns for EIR variables, starting with copying values from correponding FBP fields

fbp_cols = ['fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_',
            'fbp_ppa_id', 'fbp_exp202']

for colname_fbp in fbp_cols:
    colname_eir = colname_fbp.replace('fbp','eir')
    p10_pba50_EIR_attr[colname_eir] = p10_pba50_EIR_attr[colname_fbp]


# read p10-CoC tagging and merge into p10_pba50_EIR_attr
p10_cocid = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_cocid.csv')

# merge "coc_id" field to "p10_pba50_EIR_attr"
p10_pba50_EIR_attr = p10_pba50_EIR_attr.merge(p10_cocid, on='PARCEL_ID', how='outer')
p10_pba50_EIR_attr.rename(columns={'coc_id':'eir_coc_id'}, inplace=True)
print('p10_pba50_EIR_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_EIR_attr.shape[0],
                                                                    len(p10_pba50_EIR_attr.PARCEL_ID.unique())))

# read existing_res_bldg tagging and merge into p10_pba50_EIR_attr
p10_ex_res = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_existing_res_bldg.csv')
p10_pba50_EIR_attr = p10_pba50_EIR_attr.merge(p10_ex_res, on='PARCEL_ID', how='outer')
print('p10_pba50_EIR_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_EIR_attr.shape[0],
                                                                    len(p10_pba50_EIR_attr.PARCEL_ID.unique())))

p10_pba50_EIR_attr has 1956208 rows, 1956208 unique PARCEL_IDs
p10_pba50_EIR_attr has 1956208 rows, 1956208 unique PARCEL_IDs


In [4]:
# add "eirzoningm" field using the new concatenation

eir_zmodcat_cols = ['juris','eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 'eir_ppa_id', 'eir_exp202', 'ex_res_bldg']

p10_pba50_EIR_attr[eir_zmodcat_cols] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda x: x.fillna('NA'))
p10_pba50_EIR_attr['eirzoningm'] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
print('{} unique eirzoningmod:'.format(len(p10_pba50_EIR_attr.eirzoningm.unique())))
display(p10_pba50_EIR_attr.eirzoningm.unique())

print('p10_pba50_EIR_attr has the following fields: {}'.format(list(p10_pba50_EIR_attr)))

3535 unique eirzoningmod:


array(['livermoreNANAHRADISNANAinNA', 'livermoreNAtra3DISNANAinNA',
       'haywardNANANANANAinres', ..., 'napaNANANANANAexp1res',
       'unincorporated_napaNANANANAppainunNA',
       'unincorporated_marinNANADISNANAubz_npNA'], dtype=object)

p10_pba50_EIR_attr has the following fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area', 'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_ppa_id', 'eir_exp202', 'eir_coc_id', 'ex_res_bldg', 'eirzoningm']


In [5]:
# export
p10_pba50_EIR_attr.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_{}.csv'.format(today), index=False)